In [97]:
import pandas as pd
from owlready2 import *
import ast

In [98]:
# READ FIRST LINES FROM DOC
df = pd.read_csv('../../data/filtered_cds_data.csv', sep=',')

In [ ]:
df

In [100]:
onto = get_ontology("../../data/ontology_extended/cds-ontology_final-version_v2.owl").load()

cds = Namespace(
    world_or_ontology=onto,
    base_iri="https://constance-de-salm.de/ontology#",
    name="cds",
)

In [101]:
index = 0
for url in df['URL']:
    keywords = df['keyword_list'][index]
    if not keywords == '0':
        new_keywords = []
        for keyword in ast.literal_eval(keywords):
            keyword_id = ''.join(e for e in keyword if e.isalnum())
            if keyword_id not in onto.Automatic_Keyword.instances():
                new_keyword = onto.Automatic_Keyword(keyword_id, namespace=cds)
                new_keyword.label = keyword
                new_keywords.append(new_keyword)
            else:
                for keyword_instance in onto.Automatic_Keyword.instances():
                    if keyword_instance.iri.replace('https://constance-de-salm.de/ontology#', '') == keyword_id:
                        new_keywords.append(keyword_instance)


        for i in onto.First_Line.instances():
            fl_id = i.iri.replace('https://constance-de-salm.de/ontology#fl_', '')
            if str(fl_id) == str(url.replace('https://constance-de-salm.de/archiv/#/document/', '')):
                for keyword in new_keywords:
                    i.has_automatic_keyword.append(keyword)

    index += 1

In [102]:
onto.save(file = "../../data/ontology_extended/cds-ontology_final-version_v3.owl", format = "rdfxml")